In [1]:
import os, sys, email
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *
import pprint, re, time, pickle

pd.options.display.max_colwidth = 1000

In [2]:
### 自然言語処理
import spacy 
nlp = spacy.load("en_core_web_sm")
from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()
nlp.add_pipe(sentencizer)

In [3]:
# loadする

with open('/Users/taroaso/myprojects/OpenIE/trec/output/entity_dict.pickle', mode='rb') as f:
    entity_dict = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/entity_rows.pickle', mode='rb') as f:
    entity_rows = pickle.load(f)

In [4]:
from db import connect
engine = connect()
mail_df = pd.read_sql(sql='SELECT * FROM mail_3 ORDER BY docno ASC',con=engine, index_col=None)

In [5]:
# Bodyに含まれる文をEntity linkerにかけてEntityを取得する．
total_start = time.time()

from entityLinking import tagme

count = 0
pattern = re.compile('\W+')

for mail in mail_df.values:
    count = count + 1
    body = mail[-1]
    doc = nlp(body)

    # 意味を持たないsentenceをフィルタリングする
    sentence_list = []
    delete = 0
    for sentence in doc.sents:
        tokens = sentence.text.split()
        sentence = ' '.join(tokens)
        sentence = re.sub(r'[>|<]{1,}','',sentence)
        sentence = re.sub(r' {2,}',' ',sentence)
        if len(sentence) == 0 or sentence.isnumeric() or pattern.fullmatch(sentence):
            delete = delete + 1
            pass
        else:
            sentence_list.append(sentence)

    # フィルタリング後のsentenceをentity linkerにかける
    print('Mail:{}, delete sentences:{}'.format(count,delete))
    print("Mail:{}, left sentences:{}".format(count, len(sentence_list)))
    if len(sentence_list) == 0:
        pass
    else:
        tagme_start = time.time()
        sentences = ' '.join(sentence_list)
        json_result = tagme(sentences)
        if json_result is not None:
            linked_entities = [annotation for annotation in json_result['annotations'] if annotation['rho'] > 0.3]
            if linked_entities != []:
                for le in linked_entities:
                    spot = le['spot']
                    entity_dict.setdefault(spot,{le['title']:le['id']}) # 辞書に登録する
                    entity_rows.append((mail[0],le['title'])) # 行に追加する
    
        tagme_time = time.time() - tagme_start
        print("Mail:{}, Tagme time:{} seconds".format(count,tagme_time))

total_time = time.time() - total_start
print("Total({}mails) time:{} minutes".format(count, total_time/60))

Mail:1, delete sentences:0
Mail:1, left sentences:1
Mail:1, Tagme time:1.5670790672302246 seconds
Mail:2, delete sentences:9
Mail:2, left sentences:10
Mail:2, Tagme time:1.7839808464050293 seconds
Mail:3, delete sentences:17
Mail:3, left sentences:74
Mail:3, Tagme time:1.432610034942627 seconds
Mail:4, delete sentences:7
Mail:4, left sentences:23
Mail:4, Tagme time:1.8144092559814453 seconds
Mail:5, delete sentences:3
Mail:5, left sentences:15
Mail:5, Tagme time:1.519392967224121 seconds
Mail:6, delete sentences:45
Mail:6, left sentences:106
Mail:6, Tagme time:4.8750011920928955 seconds
Mail:7, delete sentences:38
Mail:7, left sentences:93
Mail:7, Tagme time:2.2548978328704834 seconds
Mail:8, delete sentences:42
Mail:8, left sentences:97
Mail:8, Tagme time:2.6845920085906982 seconds
Mail:9, delete sentences:43
Mail:9, left sentences:100
Mail:9, Tagme time:1.928229808807373 seconds
Mail:10, delete sentences:0
Mail:10, left sentences:191
Mail:10, Tagme time:1.793792724609375 seconds
Mail

In [11]:
# 保存しておく

with open('/Users/taroaso/myprojects/OpenIE/trec/output/entity_dict.pickle', mode='wb') as f:
    pickle.dump(entity_dict,f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/entity_rows.pickle', mode='wb') as f:
    pickle.dump(entity_rows,f)